## Day 35 Lecture 2 Assignment

In this assignment, we will combine what we have learned so far about classification algorithms this week.

In [26]:
!pip install xgboost

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
admission = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Admission_Predict.csv')

In [3]:
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In this assignment, we will predict the probability of a student being admitted to a PhD program given their stats. To make the predictions, find the median for the Chance of Admit column. Create an admit column where all probabilities above the median will receive a 1 for that column and all probabilities below the median will be a zero.

Below you will process and clean the data, try the SVM classifier, the gradient boosted decision tree classifier and XGBoost, and compare your results.

Have fun!

*Finding median to seperate Chance of Admit column*

In [4]:
admission['Chance of Admit '].describe()

count    400.000000
mean       0.724350
std        0.142609
min        0.340000
25%        0.640000
50%        0.730000
75%        0.830000
max        0.970000
Name: Chance of Admit , dtype: float64

In [5]:
# Answer below:
admission['Chance of Admit '].median()

0.73

*Creating new Admittance column based off median*

In [6]:
admission['Admit'] = admission['Chance of Admit '].apply(lambda x: 1 if x > 0.73 else 0)
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1
1,2,324,107,4,4.0,4.5,8.87,1,0.76,1
2,3,316,104,3,3.0,3.5,8.00,1,0.72,0
3,4,322,110,3,3.5,2.5,8.67,1,0.80,1
4,5,314,103,2,2.0,3.0,8.21,0,0.65,0


*EDA*

In [7]:
admission.isnull().mean()

#no nulls

Serial No.           0.0
GRE Score            0.0
TOEFL Score          0.0
University Rating    0.0
SOP                  0.0
LOR                  0.0
CGPA                 0.0
Research             0.0
Chance of Admit      0.0
Admit                0.0
dtype: float64

In [8]:
admission.shape

(400, 10)

In [9]:
#checking on value counts for Admittance
admission.Admit.value_counts()

0    209
1    191
Name: Admit, dtype: int64

In [10]:
#checking value counts for Research
admission.Research.value_counts()

1    219
0    181
Name: Research, dtype: int64

In [11]:
admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
 9   Admit              400 non-null    int64  
dtypes: float64(4), int64(6)
memory usage: 31.4 KB


*Splitting data*

In [12]:
from sklearn.model_selection import train_test_split

X = admission.drop(['Serial No.', 'Chance of Admit ', 'Admit'], axis=1)
y = admission['Admit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
X_train.shape, X_test.shape

((320, 7), (80, 7))

**Comparison of Different models**

In [16]:
from sklearn.metrics import classification_report
import time

#function
def results(model):
    time_start = time.time()
    print('Train Accuracy:', model.score(X_train, y_train), 
          '| Test Accuracy:', model.score(X_test, y_test), 
          '| Execution time: {}'.format(time.time() - time_start))
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
    display(pd.crosstab(y_test, y_pred))

*SVM classifier*

In [18]:
from sklearn.svm import SVC

svc = SVC()
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [19]:
results(svc)

Train Accuracy: 0.85625 | Test Accuracy: 0.8375 | Execution time: 0.014060735702514648
              precision    recall  f1-score   support

           0       0.87      0.85      0.86        46
           1       0.80      0.82      0.81        34

    accuracy                           0.84        80
   macro avg       0.83      0.84      0.83        80
weighted avg       0.84      0.84      0.84        80



col_0,0,1
Admit,,
0,39,7
1,6,28


*Gradient Boosted decision tree classifier*

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_iter_no_change=10)
gbc.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=10, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [25]:
results(gbc)

Train Accuracy: 0.953125 | Test Accuracy: 0.775 | Execution time: 0.005431652069091797
              precision    recall  f1-score   support

           0       0.83      0.76      0.80        46
           1       0.71      0.79      0.75        34

    accuracy                           0.78        80
   macro avg       0.77      0.78      0.77        80
weighted avg       0.78      0.78      0.78        80



col_0,0,1
Admit,,
0,35,11
1,7,27


*XGBoost*

In [27]:
from xgboost import XGBClassifier

#default parameters
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [28]:
results(xgb)

Train Accuracy: 0.9625 | Test Accuracy: 0.775 | Execution time: 0.010043621063232422
              precision    recall  f1-score   support

           0       0.85      0.74      0.79        46
           1       0.70      0.82      0.76        34

    accuracy                           0.78        80
   macro avg       0.77      0.78      0.77        80
weighted avg       0.79      0.78      0.78        80



col_0,0,1
Admit,,
0,34,12
1,6,28


*AdaBoost*

In [29]:
#change loss='exponential' in GBC for Ada 
ada = GradientBoostingClassifier(loss='exponential', n_iter_no_change=10)
ada.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='exponential', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=10, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [30]:
results(ada)

Train Accuracy: 0.971875 | Test Accuracy: 0.75 | Execution time: 0.005942583084106445
              precision    recall  f1-score   support

           0       0.81      0.74      0.77        46
           1       0.68      0.76      0.72        34

    accuracy                           0.75        80
   macro avg       0.75      0.75      0.75        80
weighted avg       0.76      0.75      0.75        80



col_0,0,1
Admit,,
0,34,12
1,8,26


*Results*

First looking at the Execution times, the Ada model and Gradient Boosting Classifier model ran the fastest with a time of 0.0059 seconds and 0.0054 seconds, respectively. Although they had the fastest times, the Recall scores for the students who were not admitted and those who were admitted were the lowest compared to the XGBoost and SVC model.

The 3 models besides SVC had high train accuracy scores at around 0.95/0.96 with test accuracy scores at around 0.75/0.77. The SVC model produced the smallest difference between train and test scores and the highest test accuracy at 0.856 and 0.837, respectively. 